In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

import requests

In [2]:
def update_progress(progress):
    bar_length = 50
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

---

In [3]:
URL = "https://de.wikipedia.org/w/api.php"

In [5]:
# request excellent arictles from german wikipedia via wiki api (10 at a time)
S = requests.Session()

params = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "*",
    "format": "json",
    "formatversion": 2,
    "srsearch": "incategory:Wikipedia:Exzellent",
    "list": "search",
    "sroffset": 0
}

response = S.get(url = URL, params = params)
data = response.json()

# get ids from excellent articles
ids = []

for entry in data["query"]["search"]:
    ids.append(entry["pageid"])

while data.get("continue"):
    params.update({"sroffset": data["continue"]["sroffset"]})
    
    #print("\n%s" % (PARAMS))
    response = S.get(url = URL, params = params)
    data = response.json()
    
    for entry in data["query"]["search"]:
        ids.append(entry["pageid"])

print("Anzahl gesammelter Exzellenter Artikel: %s" %(len(ids)))

#if DATA['query']['search'][0]['title'] == SEARCHPAGE:
#    print("Your search page '" + SEARCHPAGE + "' exists on English Wikipedia")

Anzahl gesammelter Exzellenter Artikel: 2796


In [6]:
# request data to every excellent article in german wikipedia via wikipedia api using pageid
params = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "*",
    "format": "json",
    "formatversion": 2,
    "pageids": 0
}

data = pd.DataFrame()
content = {}

for ix, id in enumerate(ids):
    update_progress(ix / len(ids))
    
    params.update({"pageids": id})
    response = S.get(url = URL, params = params)
    page = response.json()
    
    content.update({id: page["query"]["pages"][0]["revisions"][0]["slots"]["main"]["content"]})

Progress: [##################################################] 100.0%


In [7]:
# save requested data in df
df = pd.DataFrame(content.items())
df = df.rename({0: "pageid", 1:"content"}, axis = 1)

In [8]:
df

,pageid,content
0,1428,"[[Datei:Danimarca XIII secolo, plinio historia..."
1,2677,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
2,1200964,{{Dieser Artikel|behandelt den Staat. Zu weite...
3,880316,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
4,2391,{{Dieser Artikel|behandelt das Land – zu ander...
...,...,...
2777,12391297,[[Datei:Canning and Castlereagh.JPG|mini|Canni...
2778,11683048,{{Mehrere Bilder\n | Fußzeile = Explosion bei ...
2779,12170024,{{Infobox Kunstwerk\n|bilddatei = \n|bild...
2780,10278319,[[Datei:Sayyid Shaykh al-Hadi bei seiner Reise...


In [100]:
# save data to csv for faster loading
df.to_csv("./2783_excellent_article.csv")